In [73]:
# Package
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optime

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 10000

cpu


In [49]:
# Knowledge
with open('oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Character list
chars = sorted(set(text))
vocab_size = len(chars)

In [20]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s ]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encoded_hello = encode('hello')
# decoded_hello = decode(encoded_hello)
# print(decoded_hello)

# Encode (super long data)
data = torch.tensor(encode(text), dtype=torch.long)

In [22]:
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [36]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

x, y = get_batch('train')
print('input', x)
print('target', y)

tensor([168761,  85500,   7187, 193235])
input tensor([[71, 14,  0,  0, 55, 37, 69, 69],
        [69, 72, 72, 61, 62, 61,  1, 80],
        [71, 62, 80, 76, 73, 58, 73, 62],
        [ 0, 65, 58, 61,  1, 60, 72, 70]])
target tensor([[14,  0,  0, 55, 37, 69, 69, 78],
        [72, 72, 61, 62, 61,  1, 80, 66],
        [62, 80, 76, 73, 58, 73, 62, 75],
        [65, 58, 61,  1, 60, 72, 70, 62]])


In [30]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('When input is', context, 'target is', target)

When input is tensor([91]) target is tensor(48)
When input is tensor([91, 48]) target is tensor(65)
When input is tensor([91, 48, 65]) target is tensor(62)
When input is tensor([91, 48, 65, 62]) target is tensor(1)
When input is tensor([91, 48, 65, 62,  1]) target is tensor(44)
When input is tensor([91, 48, 65, 62,  1, 44]) target is tensor(75)
When input is tensor([91, 48, 65, 62,  1, 44, 75]) target is tensor(72)
When input is tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is tensor(67)


In [74]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
             # Batch, Time
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of incdices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
    
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


lY5/vH4,h#Io﻿D,D,G™Vr3emt[$xcV‘b﻿8k,G0H™p8N_P”')#I_zv,W92z4F1﻿fC-”M4H)&ge(cl.‘bv$ZX—)Fe*W6u5ofqNl."#g—8)0&gD_Xckr—#fU9'h*•g6H0Qxv#r-Wa‘_RxuD?IxcDzXp6﻿RoqXx‘W?I4ID_T$3b—WZ#2Tqjfq#I’M]LO’“dAT[&gtm”lYqj3qY”'[_z&d(T"C*[Vqx(•Roqqj ™nOJEzn:Wli•.f;﻿wm BTWgeo(IJw[n!B_“gx*AMso92o
twIFpvff•/%VPV﻿8]Vu*,Y
5Y•/‘_:6KT0yu$_j$%ZI3M$cDou“G;sJlT*EysAM-!﻿
%
)S7vTk6,pFh#2Ol6AjNrjc’ewM:v’_0B_z7Ga
VBsL#?D4ku3”mj"7_hg6,73MG%
FuZD3O VtvW?%tD6Qv#vOcX™jfW[3[E%ZWE*3gGz!Ve•﻿RW&r6’zQlb•/4a1rAN*F2(yx_’2jzv“
0OhuzwCk)3pvFh?64


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

tensor([145723,  48268,  31992, 106628])
tensor([ 56066,  38690, 120847, 144169])
tensor([105930,  10219, 139613,   4949])
tensor([142247,   8432,  48509,   6176])
tensor([140630,  68008, 189110, 131882])
tensor([106178,  69809,  49597,  60571])
tensor([155352,   9279,  45141, 194431])
tensor([ 64399, 165857,  88071, 131818])
tensor([184618,  97917, 168385,  23983])
tensor([ 21139, 133216,  52449, 129228])
tensor([178297,  71006, 140699,  54811])
tensor([102074, 141061,  81320, 158010])
tensor([ 3126, 19484, 29929, 70567])
tensor([164011,  11135,  15024,   4564])
tensor([ 62180,  99363, 119054,  12309])
tensor([ 17695,  88643,  57261, 169498])
tensor([89748, 10621, 61634, 58547])
tensor([ 70872,  11734, 196214, 153920])
tensor([ 13363, 150105, 123885,  16404])
tensor([120567, 156390,   5351, 132813])
tensor([ 42897,  62763, 111745, 163167])
tensor([ 26232,  15697, 159069,  90485])
tensor([ 39057, 148543, 175743,  45376])
tensor([ 82480,  89833, 192307, 115043])
tensor([ 16624, 121405, 